In [ ]:
!pip install cohere

In [ ]:
import cohere

import numpy as np
import warnings
warnings.filterwarnings('ignore')

key = ''
co = cohere.Client(api_key = key)

In [ ]:
!pip install tqdm

In [ ]:
!pip install pandas

In [ ]:
context = """Requirements 
Requirements engineering is concerned with discovering, eliciting, developing, analyzing, verifying, validating, communicating, documenting and managing requirements. 

Requirements construct:
Well-formed stakeholder requirements, system requirements and system element requirements shall be developed. This practice contributes to requirements validation with the stakeholders and helps ensure that the requirements accurately capture stakeholder needs. A well-formed, specified requirement contains one or more of the following:
— it shall be met or possessed by a system to solve a problem, achieve an objective or address a stakeholder concern;
— it is qualified by measurable conditions;
— it is bounded by constraints;
— it defines the performance of the system when used by a specific stakeholder or the corresponding capability of the system, but not a capability of the user, operator or other stakeholder; and
— it can be verified (e.g., the realization of the requirement in the system can be demonstrated).

A requirement is a statement that translates or expresses a need and its associated constraints and conditions. If expressed in the form of a natural language, the statement should include a subject and a verb, together with other elements necessary to adequately express the information content of the
requirement. A requirement shall state the subject of the requirement (e.g., the system, the software, etc.), what shall be done (e.g., operate at a power level, provide a field for), or a constraint on the system. Section Syntax templates for writing functional requirements, gives the syntax and examples for requirements. Condition-action tables and use cases are other means of capturing requirements.
It is important to agree in advance on the specific keywords and terms that signal the presence of a requirement. A common approach is to stipulate the following:
— Requirements are mandatory, binding provisions and use 'shall'.
— Non-requirements, such as descriptive text, use verbs such as ‘are’, ‘is’, and ‘was’. It is best to avoid using the term ‘must’, due to potential misinterpretation as a requirement.
— Statements of fact, futurity, or a declaration of purpose are non-mandatory, non-binding provisions and use 'will'. 'Will' can also be used to establish context or limitations of use.
— Preferences or goals are desired, non-mandatory, non-binding provisions and use 'should'. They are
not requirements.
— Suggestions or allowances are non-mandatory, non-binding provisions and use 'may'.
— Use positive statements and avoid negative requirements such as ‘shall not’.
— Use active voice: avoid using passive voice, such as ’it is required that’.
— Avoid using terms such as ‘shall be able to’.
All terms specific to requirements engineering should be formally defined and applied consistently throughout all requirements of the system.


ISO 29148: Syntax templates for writing functional requirements
A functional requirement can be written using one of the following templates:
Syntax for autonomous system activity: [Subject] [Signal word indicating presence of a requirement] [Action] [Constraint of action]
Example: The Invoice System [Subject] shall [Signal word indicating the presence of a requirement] display pending customer invoices [Action] in ascending order of invoice due date [Constraint of Action]
Syntax for requirment with conditions and constraints: [Condition] [Subject] [Signal word indicating the presence of a requirement] [Action] [Object] [Constraint of action]
Example: At sea state 1 [Condition], the Radar System [Subject] shall [Signal word indicating the presence of a requirement] detect [Action] targets [Object] at ranges out to 100 nautical miles [Constraint of Action]


Condition on requirement:
Conditions are measurable qualitative or quantitative attributes that are stipulated for a requirement. Conditions may limit the options open to a designer. 
For example, "When signal x is received, the system shall set the signal x received bit within 2 seconds" this requirement has a condition to take some action upon receiving the signal x.

Constraints on requirements:
Constraints are the externally imposed limitations on the system, its design or implementation, or the process used to develop or modify a system. Constraints cannot be influenced by the team members. 
Some categories of constraints include, but are not limited to:
— laws of a particular country;
— available duration or budget;
— pre-existing technology platform;
— maintenance constraints; or
— user or operator capabilities and limitations.
For example, "The car ABS development shall be completed within a timeline of 12 months" is a requirement that is limited by a timeline of 12 months.


Characteristics of individual requirements:
— Unambiguous. The requirement is stated in such a way that it can be interpreted in only one way. The requirement is stated simply and is easy to understand.
— Complete. The requirement sufficiently describes the necessary capability, characteristic, or constraint or quality factor to meet the entity need without needing other information to understand the
requirement.
— Singular. The requirement states a single capability, characteristic, constraint or quality factor.
"""

In [ ]:
import datasets
import pandas as pd
from tqdm import tqdm

location = ''

requirment_dataset_TE0 = datasets.load_from_disk(location + 'TE_0_dataset')
# requirment_dataset_TE0 = pd.DataFrame(requirment_dataset_TE0)

requirment_dataset_TE1 = datasets.load_from_disk(location + 'TE_1_dataset')
# requirment_dataset_TE1 = pd.DataFrame(requirment_dataset_TE1)

requirment_dataset_TE2 = datasets.load_from_disk(location + 'TE_2_dataset')
# requirment_dataset_TE2 = pd.DataFrame(requirment_dataset_TE2)

requirment_dataset_TE3 = datasets.load_from_disk(location + 'TE_3_dataset')
# requirment_dataset_TE3 = pd.DataFrame(requirment_dataset_TE3)

requirment_dataset_TE4 = datasets.load_from_disk(location + 'TE_4_dataset')
# requirment_dataset_TE4 = pd.DataFrame(requirment_dataset_TE4)

requirment_dataset_TENO = datasets.load_from_disk(location + 'TE_All_No_dataset')
# requirment_dataset_TENO = pd.DataFrame(requirment_dataset_TENO)

def check_requirments(requirement, requirement_type,  num_generations=1):
    
    # Search the text archive

    # Prepare the prompt
    prompt = f"""
    Excerpt from ISO 29148 titled "Systems and software engineering: requirements engineering": {context}
    Requirement: {requirement}
    Requirement Type: {requirement_type}
    
    Tasks:
    1 - Check if the given requirement uses specific keywords and terms that signal the presence of a requirement.
    2 - If the requirement type is functional, check if it follows the Syntax templates for writing functional requirements given above.
    

    Your role as a language model: ISO inspector and software requirements engineer.
    Use the tasks, first answer me '===Yes===' or '===No===', then write a short summary that explains what aspects of ISO 29148 the requirement qualifies."""

    prediction = co.generate(
        prompt=prompt,
        max_tokens=150,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )
    return prediction.generations



In [ ]:
import time

RED = "\033[31m"
GREEN = "\033[32m"
BLUE = "\033[34m"
RESET = "\033[0m"

explain_iso_dict = {}

def find_samples(dataset, condition, max_samples=20):
    count = 0
    for sample in dataset:
        if condition(sample):
            yield sample
            count += 1
            if count >= max_samples:
                return

def my_condition(sample):
    requirement_type = 'Functional'
    if sample['KM100L6V2 Label'] == 'NF':
        requirement_type = 'Non-Functional'
    results = check_requirments(sample['text'], requirement_type)
    time.sleep(12)
    if "===Yes===" in results[0]:
        explain_iso_dict[sample['text']] = handle_result(results[0])
        print(GREEN, '=============', sample['text'], '==================', RESET)
        print(GREEN, '=============', results[0], '==================', RESET)
        return True
    else:
        print(RED, '=======false========', RESET)
        return False

def handle_result(original_string):
    marker_index = original_string.find("===Yes===")
    
    if marker_index != -1:
        return original_string[marker_index + len("===Yes==="):].strip()
    else:
        print(RED, '=====Cannot find Yes====', RESET)
        print(original_string)
        return ""





In [ ]:
# 使用生成器函数查找符合条件的样本
from datasets import Dataset


# TE0
matched_samples_TE0 = find_samples(requirment_dataset_TE0, my_condition, 20)
new_samples_TE0 = [sample for sample in matched_samples_TE0]  # 从生成器中提取样本
print(new_samples_TE0)


new_dict_TE0 = {key: [dic[key] for dic in new_samples_TE0] for key in new_samples_TE0[0]}
new_dataset_TE0 = Dataset.from_dict(new_dict_TE0)
new_dataset_TE0.save_to_disk(location + 'TE0_Sampled_ISO_YES_dataset')
print(BLUE, '=================', RESET)
# print(new_dataset_TE0[0]['text'])
# print(new_dataset_TE0[1]['text'])

In [ ]:
explain_iso_dict

In [ ]:
# TE1
matched_samples_TE1 = find_samples(requirment_dataset_TE1, my_condition, 20)
new_samples_TE1 = [sample for sample in matched_samples_TE1]  # 从生成器中提取样本

new_dict_TE1 = {key: [dic[key] for dic in new_samples_TE1] for key in new_samples_TE1[0]}
new_dataset_TE1 = Dataset.from_dict(new_dict_TE1)

new_dataset_TE1.save_to_disk(location + 'TE1_Sampled_ISO_YES_dataset')
print(BLUE, '=================', RESET)
print(new_dataset_TE1)



In [ ]:
# TE2
matched_samples_TE2 = find_samples(requirment_dataset_TE2, my_condition, 20)
new_samples_TE2 = [sample for sample in matched_samples_TE2]  # 从生成器中提取样本

new_dict_TE2 = {key: [dic[key] for dic in new_samples_TE2] for key in new_samples_TE2[0]}
new_dataset_TE2 = Dataset.from_dict(new_dict_TE2)

new_dataset_TE2.save_to_disk(location + 'TE2_Sampled_ISO_YES_dataset')
print(BLUE, '=================', RESET)
print(new_dataset_TE2)



In [ ]:
# TE3
matched_samples_TE3 = find_samples(requirment_dataset_TE3, my_condition, 20)
new_samples_TE3 = [sample for sample in matched_samples_TE3]  # 从生成器中提取样本

new_dict_TE3 = {key: [dic[key] for dic in new_samples_TE3] for key in new_samples_TE3[0]}
new_dataset_TE3 = Dataset.from_dict(new_dict_TE3)

new_dataset_TE3.save_to_disk(location + 'TE3_Sampled_ISO_YES_dataset')
print(BLUE, '=================', RESET)
print(new_dataset_TE3)



In [ ]:
# TE4
matched_samples_TE4 = find_samples(requirment_dataset_TE4, my_condition, 20)
new_samples_TE4 = [sample for sample in matched_samples_TE4]  # 从生成器中提取样本

new_dict_TE4 = {key: [dic[key] for dic in new_samples_TE4] for key in new_samples_TE4[0]}
new_dataset_TE4 = Dataset.from_dict(new_dict_TE4)

new_dataset_TE4.save_to_disk(location + 'TE4_Sampled_ISO_YES_dataset')
print(BLUE, '=================', RESET)
print(new_dataset_TE4)

In [ ]:
# TENO
matched_samples_TENO = find_samples(requirment_dataset_TENO, my_condition, 20)
new_samples_TENO = [sample for sample in matched_samples_TENO]  # 从生成器中提取样本

new_dict_TENO = {key: [dic[key] for dic in new_samples_TENO] for key in new_samples_TENO[0]}
new_dataset_TENO = Dataset.from_dict(new_dict_TENO)

new_dataset_TENO.save_to_disk(location + 'TENO_Sampled_ISO_YES_dataset')
print(BLUE, '=================', RESET)
print(new_dataset_TENO)

In [ ]:
explain_dic = Dataset.from_dict(explain_iso_dict)
explain_dic.save_to_disk(location + 'Explain_ISO_dataset')

In [ ]:
# responses = []

# # for requirement in tqdm(requirment_dataset_TE0):
# num_no = 0
# num_yes = 0

# for index,row in requirment_dataset_TE0.iterrows():
#     requirement_type = 'Functional'
#     if row['KM100L6V2 Label'] == 'NF':
#         requirement_type = 'Non-Functional'
#     results = check_requirments(row['text'], requirement_type)
#     if "===Yes===" in results[0]
        
#     print(results[0])
#     break
# for requirement in requirment_dataset_TE0:
#     requirement_type = 'Functional'
#     display(requirement)
#     if requirement['KM100L6V2 Label'].str == 'NF':
#         requirement_type = 'Non-Functional'
#     results = check_requirments(requirement['text'], requirement_type)
#     # output = io.StringIO()
#     # sys.stdout = output
#     print(results[0])
#     break
#     # printed_result = output.getvalue()
#     # responses.append(printed_result)
#     # time.sleep(30)

In [ ]:
str = '''
Prompt:

Excerpt from ISO 29148 titled "Systems and software engineering: requirements engineering": {context}

Requirement: NPAC SMS shall send a notification to the Service Provider (new or old) who has not yet 
acknowledged the cancel pending status when the Cancellation-Initial Concurrence Window tunable parameter expires.

Requirement Type: Functional Requirement

Tasks:
1 - Check if the given requirement uses specific keywords and terms that signal the presence of a requirement.
2 - Check if it follows the Syntax templates for writing requirements given above.


Your role as a language model: ISO inspector and software requirements engineer.
Use the tasks, first answer me '===Yes===' or '===No===', then write a short summary that explains what aspects of 
ISO 29148 the requirement qualifies.




Answer:
===Yes=== 

The requirement provided qualifies as a Functional Requirement of ISO 29148 as it states a specific action (send a noti
fication) to be performed by the NPAC SMS system upon a particular condition (when the Cancellation-Initial Concurrence
Window tunable parameter expires). It is unambiguous, complete, and singular, and uses the signal word 'shall' to indic
ate a requirement. 

The requirement is written using the syntax template for functional requirements: [Condition] [Subject] [Signal word] 
[Action] [Object] [Constraint of Action], with the condition, subject, action, and object being clearly defined. It is 
also bounded by a constraint, which is the time-sensitive nature of the requirement. 

'''

print(str)